# Proyecto Sprint 14

Para este proyecto trabajaremos un caso practico de la empresa Rusty Bargain que se dedica a la venta de autos usados, que esta desarrollando una aplicacion para averiguar rapidamente el valor en el mercado de un auto. 
Tenemos acceso a una base de datos que tiene informacion de 356,000 autos en donde podemos encotrar la marca, el precio, tipo de gasolina que usa entre otras caracteristicas.

Objetivo: Revisar entre los diferentes Modelos de regresion como Regresion Lineal, Bosque aleatorio y otros cual tiene mejor prediccion usando la metrica de Raiz del Error cuadratico Medio y tambien considerando los tiempos de entrenamiento y prediccion.

## Preprocesamiento de datos 

In [ ]:
# Importacion de librerias 
import pandas as pd 
import numpy as np
import math 
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
import os
import time

#### Analisis Exploratorio de datos 

In [3]:
# Vista general del dataset
df = pd.read_csv('data/car_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [ ]:
# Revisar valores nulos en el dataset
print("Valores nulos por columna:")
print(df.isnull().sum())

In [ ]:
# Porcentaje de valores nulos por columna
print("\nPorcentaje de valores nulos por columna:")
print((df.isnull().sum() / len(df)) * 100)

Observamos que las columnas que tiene valores faltantes son 'VehicleType', 'Gearbox', 'Model', 'FuelType' y 'Notrepaired'

In [ ]:
# Mostrar tipos de datos de cada columna
print("Tipos de datos por columna:")
print(df.dtypes)

# Revisar algunas filas para entender los datos
print("\nPrimeras 5 filas del dataset:")
print(df.head())

Observamos que tenemos caracteristicas categoricas y numericas por lo que despues tendremos que codificarlas o escalarlas para que se puedan entrenar mejor nuestros modelos

Intentamos usar K-NearestNeighbor para la imputacion de datos ya que analizando podemos saber que un tipo de vehiculo como las SUV que regularmente tiene una caja automatica, con esta idea este metodo nos podra ayudar a hacer una suposicion mas acercada a la realidad y el modelo pueda entrenarse mejor lamentablemente por la magnitud del dataset toma demasiados recursos hacer esta imputacion por lo que optaremos por rellenar con unknow los datos categoricos faltantes, esta parte creemos que es beneficiosa ya que en un escenario de practica real puede que igual falten datos y aun asi queremos saber una aproximacion del precio 

#### Imputacion de datos

In [ ]:
# Rellenar valores nulos en columnas categóricas con 'unknown'
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']
for col in categorical_columns:
    df[col].fillna('unknown', inplace=True)

# Verificar que ya no hay nulos
print("Valores nulos tras rellenar con 'unknown':")
print(df.isnull().sum())

#### Seleccion de Caracteristicas Relevantes

In [ ]:
# Eliminar columnas no relevantes para la predicción
columns_to_drop = ['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode']
df = df.drop(columns=columns_to_drop)

# Mostrar columnas restantes
print("Columnas seleccionadas:")
print(df.columns)

#### Escalamiento y codificacion de datos

In [ ]:
# Mantener el dataset original con categorías en texto para CatBoost
df_catboost = df.copy()
print("Dataset para CatBoost (categorías sin codificar):")
print(df_catboost.head())

# Crear una versión codificada para otros modelos
df_encoded = df.copy()
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired', 'Brand']

# Aplicar LabelEncoder a las columnas categóricas
le = LabelEncoder()
for col in categorical_columns:
    df_encoded[col] = le.fit_transform(df_encoded[col])

# Mostrar primeras filas del dataset codificado
print("\nDataset codificado para otros modelos (Regresión Lineal, XGBoost, etc.):")
print(df_encoded.head())

## Entrenamiento de modelos de regresion 

In [ ]:
# Definir las caracteristicas y objetivo en el df_encoded
X_encoded = df_encoded.drop(columns=['Price'])
y = df_encoded['Price']

# Definir las caracteristicas y objetivo en df_catboost
X_catboost = df_catboost.drop(columns=['Price'])

# Lista para almacenar tiempos y métricas
results = []

In [ ]:
# Dividir en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
# Usamos una sola división para mantener los índices consistentes
train_idx, test_idx = train_test_split(range(len(y)), test_size=0.2, random_state=42)

# Aplicar los índices a ambos conjuntos de características y objetivos 
X_train_enc, X_test_enc = X_encoded.iloc[train_idx], X_encoded.iloc[test_idx]
X_train_cat, X_test_cat = X_catboost.iloc[train_idx], X_catboost.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [ ]:
# Entrenar regresión lineal
start_time_train = time.time()
lr_model = LinearRegression()
lr_model.fit(X_train_enc, y_train)
training_time_lr = time.time() - start_time_train

# Predecir y calcular tiempo de predicción
start_time_pred = time.time()
y_pred_lr = lr_model.predict(X_test_enc)
prediction_time_lr = time.time() - start_time_pred
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

# Guardar resultados
results.append({
    'Model': 'Linear Regression',
    'Training Time': training_time_lr,
    'Prediction Time': prediction_time_lr,
    'RMSE': rmse_lr
})
print(f"Regresión Lineal - Tiempo de entrenamiento: {training_time_lr:.2f}s, Tiempo de predicción: {prediction_time_lr:.2f}s, RECM: {rmse_lr:.2f}")

In [ ]:
# Configuraciones de hiperparámetros para Bosque aleatorio
rf_configs = [
    {'n_estimators': 50, 'max_depth': 10, 'random_state': 42},
    {'n_estimators': 100, 'max_depth': 20, 'random_state': 42},
    {'n_estimators': 200, 'max_depth': 30, 'random_state': 42}
]

# Entrenar y evaluar cada configuración# Entrenar y evaluar cada configuración
for i, config in enumerate(rf_configs, 1):
    start_time_train = time.time()
    rf_model = RandomForestRegressor(**config)
    rf_model.fit(X_train_enc, y_train)
    training_time_rf = time.time() - start_time_train
    
    start_time_pred = time.time()
    y_pred_rf = rf_model.predict(X_test_enc)
    prediction_time_rf = time.time() - start_time_pred
    rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
    
    results.append({
        'Model': f'Random Forest {i}',
        'Training Time': training_time_rf,
        'Prediction Time': prediction_time_rf,
        'RMSE': rmse_rf
    })
    print(f"Bosque Aleatorio {i} - Tiempo de entrenamiento: {training_time_rf:.2f}s, Tiempo de predicción: {prediction_time_rf:.2f}s, RECM: {rmse_rf:.2f}")

In [ ]:
# Configuraciones de hiperparámetros para LightGBM
lgb_configs = [
    {'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.1, 'random_state': 42},
    {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.05, 'random_state': 42},
    {'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.01, 'random_state': 42}
]

# Entrenar y evaluar cada configuración
for i, config in enumerate(lgb_configs, 1):
    start_time_train = time.time()
    lgb_model = lgb.LGBMRegressor(**config)
    lgb_model.fit(X_train_enc, y_train)
    training_time_lgb = time.time() - start_time_train
    
    start_time_pred = time.time()
    y_pred_lgb = lgb_model.predict(X_test_enc)
    prediction_time_lgb = time.time() - start_time_pred
    rmse_lgb = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
    
    results.append({
        'Model': f'LightGBM {i}',
        'Training Time': training_time_lgb,
        'Prediction Time': prediction_time_lgb,
        'RMSE': rmse_lgb
    })
    print(f"LightGBM {i} - Tiempo de entrenamiento: {training_time_lgb:.2f}s, Tiempo de predicción: {prediction_time_lgb:.2f}s, RECM: {rmse_lgb:.2f}")

In [ ]:
# Crear un directorio temporal si no existe
temp_dir = 'C:/Users/Usuario/temp_catboost'  # Cambia a '/tmp' en plataformas Unix si es necesario
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

# Identificar columnas categóricas para CatBoost
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired', 'Brand']

# Configuraciones de hiperparámetros con directorio explícito
cat_configs = [
    {'iterations': 100, 'depth': 6, 'learning_rate': 0.1, 'random_seed': 42, 'logging_level': 'Silent', 'save_snapshot': False, 'train_dir': temp_dir},
    {'iterations': 200, 'depth': 8, 'learning_rate': 0.05, 'random_seed': 42, 'logging_level': 'Silent', 'save_snapshot': False, 'train_dir': temp_dir},
    {'iterations': 300, 'depth': 10, 'learning_rate': 0.01, 'random_seed': 42, 'logging_level': 'Silent', 'save_snapshot': False, 'train_dir': temp_dir}
]

# Entrenar y evaluar cada configuración
for i, config in enumerate(cat_configs, 1):
    start_time_train = time.time()
    cat_model = CatBoostRegressor(**config)
    cat_model.fit(X_train_cat, y_train, cat_features=cat_features)
    training_time_cat = time.time() - start_time_train
    
    start_time_pred = time.time()
    y_pred_cat = cat_model.predict(X_test_cat)
    prediction_time_cat = time.time() - start_time_pred
    rmse_cat = np.sqrt(mean_squared_error(y_test, y_pred_cat))
    
    results.append({
        'Model': f'CatBoost {i}',
        'Training Time': training_time_cat,
        'Prediction Time': prediction_time_cat,
        'RMSE': rmse_cat
    })
    print(f"CatBoost {i} - Tiempo de entrenamiento: {training_time_cat:.2f}s, Tiempo de predicción: {prediction_time_cat:.2f}s, RECM: {rmse_cat:.2f}")

In [ ]:
# Configuraciones de hiperparámetros para XGBoost
xgb_configs = [
    {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'random_state': 42},
    {'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.05, 'random_state': 42},
    {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.01, 'random_state': 42}
]

# Entrenar y evaluar cada configuración
for i, config in enumerate(xgb_configs, 1):
    start_time_train = time.time()
    xgb_model = xgb.XGBRegressor(**config)
    xgb_model.fit(X_train_enc, y_train)
    training_time_xgb = time.time() - start_time_train
    
    start_time_pred = time.time()
    y_pred_xgb = xgb_model.predict(X_test_enc)
    prediction_time_xgb = time.time() - start_time_pred
    rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
    
    results.append({
        'Model': f'XGBoost {i}',
        'Training Time': training_time_xgb,
        'Prediction Time': prediction_time_xgb,
        'RMSE': rmse_xgb
    })
    print(f"XGBoost {i} - Tiempo de entrenamiento: {training_time_xgb:.2f}s, Tiempo de predicción: {prediction_time_xgb:.2f}s, RECM: {rmse_xgb:.2f}")

## Analisis del modelo 

In [ ]:
# Convertir results a DataFrame
results_df = pd.DataFrame(results)

# Ordenar por RECM (calidad de predicción) de menor a mayor
results_df = results_df.sort_values(by='RMSE')

# Mostrar tabla
print("Tabla comparativa de modelos:")
print(results_df)

In [ ]:
# Ordenar por Training Time de menor a mayor
results_df = results_df.sort_values(by='Training Time')

# Mostrar tabla
print("Tabla comparativa de modelos:")
print(results_df)

In [ ]:
# Ordenar por RPrediction Time de menor a mayor
results_df = results_df.sort_values(by='Prediction Time')

# Mostrar tabla
print("Tabla comparativa de modelos:")
print(results_df)

# Conclusiones 

De acuerdo al objetivo donde es escoger un modelo que tenga buena prediccion, rapida velocidad de entrenamiento y rapida velocidad de predicion podemos optar por un modelo XGBoost donde la metrica RECM tiene de las mas bajas diferencia entre lo predicho y lo real y donde el tiempo entrenamiento y prediccion no son tan altos como en un Random Forest, la desicion de tomar un modelo y otro dependera de los recursos con los que contamos como equipo y tiempo asi como el objetivo.